## Importing Libraries

In [3]:
from segysak.segy import segy_loader,segy_header_scan, segy_header_scrape
# from pyts.decomposition import SingularSpectrumAnalysis
# from plotting_functions import (plot_single_trace_fig, wigle_plot)
from warnings import filterwarnings
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
from os.path import join, isdir, expanduser
from os import listdir
from torch import from_numpy

In [4]:
filterwarnings('ignore')

## Selecting the Data

In [6]:
directory = 'D:\Dados_TCC\Dados_pred'

files = listdir(directory)
for i in range(len(files)):
    if isdir(join(directory,files[i])):
        print(f'Index:{i}, Directory: {files[i]}')
    else:
        print(f'Index:{i}, File: {files[i]}')

Index:0, File: CREWS_Completo_LMO_semAGC.sgy
Index:1, File: CREWS_Completo_OFF-0-1500_LMO_semAGC.sgy
Index:2, File: CREWS_Completo_OFF-0-1500_semAGC.sgy
Index:3, File: CREWS_Completo_semAGC.sgy
Index:4, File: Crews_Geometria_Completo_Para_Pick_Romulo.sgy
Index:5, File: dataset_predict.npz
Index:6, File: pred_dataset.csv


In [7]:
#Choose the index
index = 3

#Loading the data
data =  segy_loader(join(directory,files[index])) #Carregamento da sísmica
header = segy_header_scrape(join(directory,files[index])) #carregamento do header

  0%|          | 0.00/238k [00:00<?, ? traces/s]

                        byte_loc     count          mean            std   
TRACE_SEQUENCE_LINE            1  237687.0  1.188440e+05   68614.471054  \
TRACE_SEQUENCE_FILE            5  237687.0  1.188440e+05   68614.471054   
FieldRecord                    9  237687.0  1.957860e+02     103.643798   
TraceNumber                   13  237687.0  3.449758e+02     198.885497   
EnergySourcePoint             17  237687.0  1.957860e+02     103.643798   
CDP                           21  237687.0  7.525560e+03    2224.293518   
offset                        37  237687.0  1.739022e+03     869.420928   
ReceiverGroupElevation        41  237687.0  9.300565e+06  173978.298122   
SourceSurfaceElevation        45  237687.0  9.298599e+06  113858.111567   
SourceX                       73  237687.0  3.474717e+07   71473.178491   
SourceY                       77  237687.0  5.647136e+08   63051.523518   
GroupX                        81  237687.0  3.477524e+07   87088.769446   
GroupY                   

Converting SEGY:   0%|          | 0.00/238k [00:00<?, ? traces/s]

  0%|          | 0.00/238k [00:00<?, ? traces/s]

In [8]:
#Obtendo os números dos traços de cada shot gather
shoot_number = header.FieldRecord.unique()
# print(f'Número do tiro: {shoot_number}')
shoot_index = [(header[header.FieldRecord == i]['TRACE_SEQUENCE_FILE'].min()-1,
header[header.FieldRecord == i]['TRACE_SEQUENCE_FILE'].max()-1) for i in header.FieldRecord.unique()]
# print('')
# print(f'Index do tiro: {shoot_index}')

In [9]:
## Resumindo
#As amostras do tiro vão da linha tal até linha tal
for i in range(len(shoot_number)):
    print(f'Número do tiro: {shoot_number[i]}, Index: {shoot_index[i][0]} até {shoot_index[i][1]}')
    print('')

Número do tiro: 18, Index: 0 até 689

Número do tiro: 19, Index: 690 até 1373

Número do tiro: 20, Index: 1374 até 2063

Número do tiro: 21, Index: 2064 até 2753

Número do tiro: 22, Index: 2754 até 3443

Número do tiro: 23, Index: 3444 até 4133

Número do tiro: 24, Index: 4134 até 4823

Número do tiro: 25, Index: 4824 até 5513

Número do tiro: 26, Index: 5514 até 6203

Número do tiro: 27, Index: 6204 até 6893

Número do tiro: 28, Index: 6894 até 7583

Número do tiro: 29, Index: 7584 até 8273

Número do tiro: 30, Index: 8274 até 8963

Número do tiro: 32, Index: 8964 até 9650

Número do tiro: 33, Index: 9651 até 10337

Número do tiro: 34, Index: 10338 até 11024

Número do tiro: 35, Index: 11025 até 11711

Número do tiro: 36, Index: 11712 até 12398

Número do tiro: 37, Index: 12399 até 13085

Número do tiro: 38, Index: 13086 até 13772

Número do tiro: 39, Index: 13773 até 14459

Número do tiro: 40, Index: 14460 até 15146

Número do tiro: 41, Index: 15147 até 15833

Número do tiro: 42, In

In [10]:
#pegando todos os valores das amostras e transformando em DataFrame
X = pd.DataFrame(data.data.values)

#Criando uma coluna nula n_tiro preenchida por zeros
X['N_tiro'] = 0

#Substituindo os valores da coluna n_tiro pelo numero do tiro respectivo, baseado pelo index
for i,j in zip(shoot_number,shoot_index):
    X['N_tiro'].loc[j[0]:j[1]+1] = i

In [11]:
X

,0,1,2,3,4,5,6,7,8,9,...,992,993,994,995,996,997,998,999,1000,N_tiro
0,-4.768662e-07,-5.960828e-07,1.192166e-07,5.960828e-07,-1.192166e-07,-7.152993e-07,-1.192166e-07,5.960828e-07,4.768662e-07,-2.384331e-07,...,2.095868e-08,5.029126e-08,-8.708855e-08,-2.696396e-07,-6.005286e-08,-5.988536e-07,-5.522947e-07,8.847627e-08,-4.261002e-07,18
1,2.375194e-07,-1.187597e-07,2.375194e-07,4.750387e-07,0.000000e+00,2.375194e-07,1.187597e-07,2.375194e-07,3.562791e-07,-2.375194e-07,...,-2.793960e-07,-4.940434e-07,2.421344e-07,-3.255137e-07,-5.401612e-07,-6.146843e-08,-9.781155e-08,-1.941798e-07,-2.863816e-07,18
2,4.768662e-07,1.430599e-06,4.768662e-07,1.192166e-07,2.503548e-06,4.291796e-06,4.291796e-06,4.768663e-06,6.318477e-06,8.464375e-06,...,7.289812e-06,2.720513e-06,2.068888e-07,3.073508e-06,7.229784e-06,8.485464e-06,4.380154e-06,1.544116e-07,-1.021940e-08,18
3,8.372444e-08,8.372444e-08,8.372444e-08,0.000000e+00,-8.372444e-08,0.000000e+00,0.000000e+00,-8.372444e-08,-8.372444e-08,-8.372444e-08,...,3.939321e-07,3.492270e-07,2.980033e-07,2.300441e-07,2.649483e-07,1.932444e-07,1.173336e-07,1.588025e-07,6.425353e-08,18
4,-2.102857e-07,2.102857e-07,-1.051429e-07,-3.154286e-07,-2.102857e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.102857e-07,...,-1.950958e-07,-7.452763e-08,4.018773e-07,1.075786e-07,-9.639439e-08,1.573807e-07,2.067313e-07,2.873275e-07,2.160618e-07,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237682,1.844884e-08,-9.934549e-09,-4.162945e-08,-4.115554e-08,-3.264126e-08,-2.980219e-08,-2.270671e-08,-6.149126e-09,3.311517e-09,4.724487e-10,...,-1.972721e-07,-3.065569e-07,-3.637756e-07,-3.519491e-07,-2.829039e-07,-1.769070e-07,-3.312405e-08,1.329378e-07,2.824132e-07,375
237683,2.833051e-09,7.083592e-09,1.511057e-08,7.555769e-09,2.360876e-09,1.416526e-09,-1.086099e-08,-1.747241e-08,-1.274969e-08,-7.083592e-09,...,5.515437e-07,4.150648e-07,2.578521e-07,1.057867e-07,-2.785380e-08,-1.128584e-07,-1.369598e-07,-1.048246e-07,-2.172894e-08,375
237684,1.890007e-09,-4.724182e-09,-2.835011e-09,-1.181087e-08,-1.511839e-08,-3.779178e-09,-2.835011e-09,-2.835011e-09,-1.086587e-08,-1.417338e-08,...,6.080153e-07,4.989147e-07,3.340115e-07,1.483656e-07,-2.835011e-09,-1.289930e-07,-2.026561e-07,-2.102161e-07,-1.634621e-07,375
237685,-2.079480e-08,-3.024707e-08,-3.166476e-08,-1.134253e-08,-2.362825e-09,9.451302e-10,2.363019e-08,2.504788e-08,1.748684e-08,1.228766e-08,...,4.650560e-07,4.300894e-07,3.473767e-07,2.178753e-07,6.568729e-08,-9.074671e-08,-2.084260e-07,-2.528419e-07,-2.287488e-07,375


In [9]:
tiros = []
for i in X.loc[:,'N_tiro']:
    tiros.append(i)
print(tiros)

[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,

In [10]:
valores = np.unique(tiros)
valores

array([ 18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,
        45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,
        58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
        71,  72,  73,  74,  76,  77,  78,  79,  80,  81,  82,  83,  84,
        85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
        98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110,
       111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123,
       124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136,
       137, 138, 139, 140, 141, 142, 143, 144, 145, 147, 148, 149, 150,
       151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163,
       164, 165, 166, 167, 168, 169, 170, 171, 172, 174, 175, 176, 177,
       179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191,
       192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 20

In [11]:
dataset = X[X.N_tiro.isin(valores)]

In [12]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,992,993,994,995,996,997,998,999,1000,N_tiro
0,-4.768662e-07,-5.960828e-07,1.192166e-07,5.960828e-07,-1.192166e-07,-7.152993e-07,-1.192166e-07,5.960828e-07,4.768662e-07,-2.384331e-07,...,2.095868e-08,5.029126e-08,-8.708855e-08,-2.696396e-07,-6.005286e-08,-5.988536e-07,-5.522947e-07,8.847627e-08,-4.261002e-07,18
1,2.375194e-07,-1.187597e-07,2.375194e-07,4.750387e-07,0.000000e+00,2.375194e-07,1.187597e-07,2.375194e-07,3.562791e-07,-2.375194e-07,...,-2.793960e-07,-4.940434e-07,2.421344e-07,-3.255137e-07,-5.401612e-07,-6.146843e-08,-9.781155e-08,-1.941798e-07,-2.863816e-07,18
2,4.768662e-07,1.430599e-06,4.768662e-07,1.192166e-07,2.503548e-06,4.291796e-06,4.291796e-06,4.768663e-06,6.318477e-06,8.464375e-06,...,7.289812e-06,2.720513e-06,2.068888e-07,3.073508e-06,7.229784e-06,8.485464e-06,4.380154e-06,1.544116e-07,-1.021940e-08,18
3,8.372444e-08,8.372444e-08,8.372444e-08,0.000000e+00,-8.372444e-08,0.000000e+00,0.000000e+00,-8.372444e-08,-8.372444e-08,-8.372444e-08,...,3.939321e-07,3.492270e-07,2.980033e-07,2.300441e-07,2.649483e-07,1.932444e-07,1.173336e-07,1.588025e-07,6.425353e-08,18
4,-2.102857e-07,2.102857e-07,-1.051429e-07,-3.154286e-07,-2.102857e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.102857e-07,...,-1.950958e-07,-7.452763e-08,4.018773e-07,1.075786e-07,-9.639439e-08,1.573807e-07,2.067313e-07,2.873275e-07,2.160618e-07,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237682,1.844884e-08,-9.934549e-09,-4.162945e-08,-4.115554e-08,-3.264126e-08,-2.980219e-08,-2.270671e-08,-6.149126e-09,3.311517e-09,4.724487e-10,...,-1.972721e-07,-3.065569e-07,-3.637756e-07,-3.519491e-07,-2.829039e-07,-1.769070e-07,-3.312405e-08,1.329378e-07,2.824132e-07,375
237683,2.833051e-09,7.083592e-09,1.511057e-08,7.555769e-09,2.360876e-09,1.416526e-09,-1.086099e-08,-1.747241e-08,-1.274969e-08,-7.083592e-09,...,5.515437e-07,4.150648e-07,2.578521e-07,1.057867e-07,-2.785380e-08,-1.128584e-07,-1.369598e-07,-1.048246e-07,-2.172894e-08,375
237684,1.890007e-09,-4.724182e-09,-2.835011e-09,-1.181087e-08,-1.511839e-08,-3.779178e-09,-2.835011e-09,-2.835011e-09,-1.086587e-08,-1.417338e-08,...,6.080153e-07,4.989147e-07,3.340115e-07,1.483656e-07,-2.835011e-09,-1.289930e-07,-2.026561e-07,-2.102161e-07,-1.634621e-07,375
237685,-2.079480e-08,-3.024707e-08,-3.166476e-08,-1.134253e-08,-2.362825e-09,9.451302e-10,2.363019e-08,2.504788e-08,1.748684e-08,1.228766e-08,...,4.650560e-07,4.300894e-07,3.473767e-07,2.178753e-07,6.568729e-08,-9.074671e-08,-2.084260e-07,-2.528419e-07,-2.287488e-07,375


In [13]:
# dataset.to_csv(join(directory,'pred_dataset.csv'), index=False)

In [19]:
#Pegando colunas baseadas pelo index da coluna
X = np.asarray(dataset[np.arange(0,1001)])

In [20]:
X.shape

(237687, 1001)

In [22]:
inputs_pred = [np.asarray(X)[i] for i in range(X.shape[0])]
print(f'Total de traços: {len(inputs_pred)}')
X_pred = np.asarray(inputs_pred)
print(f'Inputs Pred Shape: {X_pred.shape}')

Total de traços: 237687
Inputs Pred Shape: (237687, 1001)


## Saving Data

In [ ]:
np.savez_compressed(join(directory,'dataset_predict.npz'), arr_0 = X_pred)

## Appliyng SSA (Singular Spectrum Analysis)

In [16]:
from sklearn.preprocessing import minmax_scale

In [17]:
#Normalizing Data
X_pred.std(axis=1).shape
std_corr = np.where(X_pred.std(axis=1)==0,1,X_pred.std(axis=1))
#X_pred_norm = (X_pred.T/std_corr).T
#X_pred_norm = (X_pred.T/X_pred.std(axis=1)).T
X_pred_norm = minmax_scale(np.asarray(dataset[np.arange(0,1001)]).T,feature_range=(-1,1)).T 
X_pred_norm = np.float32(X_pred_norm)

In [18]:
# We decompose the time series into three subseries
window_size = 15
groups = [np.arange(i, i + 5) for i in range(0, 11, 5)]

In [19]:
# Singular Spectrum Analysis
ssa = SingularSpectrumAnalysis(window_size=15, groups=groups)
X_pred_ssa = ssa.fit_transform(X_pred_norm[10000:20000])
print(X_pred_ssa.shape)

(10000, 3, 1001)


In [23]:
np.savez_compressed(join(directory,'dataset_predict.npz'), arr_0 = X_pred)